# Atmospheric Correction (example 1)
*---*

### THE PROBLEM

A common problem in remote sensing is the need to to convert at-sensor radiance (L) to surface reflectance (ρ). One way to describe the relationship between the two is:

ρ = π(L -Lp) / τ(Edir + Edif)

where

* ρ = surface reflectance
* π = 3.1415..
* L = at-sensor radiance
* Lp = path radiance
* τ = transmissivity (from surface to satellite)
* Edir = direct solar irradiance 
* Edif = diffuse solar irradiance



Let's say we know the at-sensor radiance

In [23]:
L = 100

We now have 2 knowns (i.e. π and L) and need the remaining 4 unknowns (i.e. Edir, Edif, τ and Lp) to calculate surface reflectance. 

### THE SOLUTION

The 4 unknowns depend on atmospheric conditions and Earth-Sun-Satellite geometry

**Atmospheric Conditions** <br />
For visible through short-wave infrared wavelengths (0.4 to 2.5 microns) the key variables are:

* water vapour (g cm<sup>-2</sup>)
* ozone (atm-cm)
* aerosol optical thickness

In [24]:
H2O = 1
O3 = 0.4
AOT = 0.3

**Earth-Sun-Satellite Geometry** <br />

* target altitude (km)
* solar zenith angle (degrees)
* view zentith angle (degrees)
* Earth-Sun distance (Astronomical Units)

Instead of Earth-Sun distance we use day-of-year to correct for the eccentricity in the Earth's orbit around the sun. See [here](https://github.com/samsammurphy/atmcorr/wiki/Elliptical-Orbit-Correction) for details.

In [25]:
alt = 0
solar_z = 20
view_z = 0   
doy = 4

**Where might we get these values from?**

Potential sources include:<br>

* Water vapour: [NCEP/NCAR](http://journals.ametsoc.org/doi/abs/10.1175/1520-0477%281996%29077%3C0437%3ATNYRP%3E2.0.CO%3B2)
* Ozone: [TOMS/OMI](http://ozoneaq.gsfc.nasa.gov/missions). 
* Aerosol optical thickness: [MODIS Aerosol Product](http://modis-atmos.gsfc.nasa.gov/MOD04_L2/index.html) or in-scene techniques
* Geometry and day-of-year: image metadata


**How to convert at-sensor radiance to surface reflectance?**

We can use an interpolated lookup table (iLUT). Let's say we want to correct Landsat 8's blue band (i.e. B2) using a continental (CO) aerosol model.

First, open the relevant iLUT:

In [26]:
import pickle

iLUT_path = '/home/sam/git/atmcorr/iLUTs/LANDSAT_OLI_MA/viewz_0/LANDSAT_OLI_MA_0_B2.ilut'
with open(iLUT_path, 'rb') as f:
    iLUT = pickle.load(f)

Then retrieve outputs at perihelion:

In [27]:
Edir, Edif, tau2, Lp = iLUT(solar_z,H2O,O3,AOT,alt)

correct for the Earth's elliptical orbit

In [34]:
import numpy as np

def elliptical_correction(doy):
    a = 0.03275
    b = 18.992
    c = 0.968047
    return a*np.cos(doy/(b*math.pi)) + c

ec = elliptical_correction(doy)

Edir = Edir*ec
Edif = Edif*ec
Lp = Lp*ec

calculate surface reflectance:

In [36]:
import math
ρ = (math.pi*(L-Lp))/(tau2*(Edir+Edif))

print('Surface Reflectance = {:.3f}'.format(ρ))

Surface Reflectance = 0.099


**DONE!**